# AiiDA 
This implementation uses AiiDA to implement the equation of state workflow with Quantum ESPRESSO.
It intentionally provides a minimal implementation for the sake of simplicity:

* Quantum ESPRESSO is run locally, but could be run on a remote cluster
* The entire procedure is not yet wrapped up in a single workflow
* The SCF calculations are run serially but could be run in parallel

## Installation and setup
```
pip install aiida-shell
verdi presto
```


In [1]:
import pathlib
import tempfile
import typing as t
import matplotlib.pyplot as plt
from ase.build import bulk
from ase.io import read, write
from aiida import engine, orm, load_profile
from aiida_shell import launch_shell_job

In [2]:
FILEPATH_PSEUDOS = pathlib.Path.cwd() / 'espresso' / 'pseudo'

@engine.calcfunction
def generate_scaled_structures(structure: orm.StructureData, scaling_factors: list[float]) -> dict[str, orm.StructureData]:
    scaled_structures = {}

    for index, scaling_factor in enumerate(scaling_factors):
        atoms = structure.get_ase()
        atoms.set_cell(atoms.get_cell() * scaling_factor, scale_atoms=True)
        scaled_structures[f's_{index}'] = orm.StructureData(ase=atoms)

    return scaled_structures

@engine.calcfunction
def create_pw_input_file(structure: orm.StructureData, inputs: dict[str, t.Any]) -> str:
    with tempfile.NamedTemporaryFile(suffix='.pwi', mode='w+') as handle:
        write(
            filename=handle.name,
            images=[structure.get_ase()],
            **inputs.get_dict()
        )
        return orm.SinglefileData(handle)

@engine.calcfunction
def parse_pw_output_file(output: orm.SinglefileData) -> dict[str, orm.StructureData | float]:
    with output.as_path() as filename:
        atoms = read(filename, format='espresso-out')
    return {'structure': orm.StructureData(ase=atoms), 'energy': orm.Float(atoms.calc.results['energy'])}

@engine.workfunction
def run_pw(structure: orm.StructureData, inputs: dict[str, t.Any]) -> dict[str, orm.StructureData | float]:
    script = create_pw_input_file(structure, inputs)

    results, _ = launch_shell_job(
        'pw.x',
        arguments='-in {script}',
        nodes={
            'script': script,
        },
        metadata={
            'options': {
                'prepend_text': f'export ESPRESSO_PSEUDO {FILEPATH_PSEUDOS.as_posix()}'
            }
        }
    )

    return parse_pw_output_file(results['stdout'])

@engine.calcfunction
def plot_energy_volume_curve(*args):
    """Function to plot the final graph"""

    def batched(iterable, size):
        """This is essentially ``itertools.batched`` but only exists for Python 3.12 and later."""
        import itertools
        iterator = iter(iterable)
        while batch := tuple(itertools.islice(iterator, size)):
            yield batch

    structures, energies = zip(*batched(args, 2))
    volumes = [structure.get_ase().get_volume() for structure in structures]

    plt.plot(volumes, energies)
    plt.xlabel('Volume')
    plt.ylabel('Energy')
    plt.savefig('evcurve.png')
    return orm.SinglefileData(pathlib.Path.cwd() / 'evcurve.png')

In [3]:
load_profile()
atoms = bulk('Al', a=4.05, cubic=True)
scaling_factors = [0.9, 0.95, 1.0, 1.05, 1.1]
inputs = {
    'Crystal': True,
    'kpts': (1, 1, 1),
    'input_data': {
        'calculation': 'relax',
        'occupations': 'smearing',
        'degauss': 0.02,
    },
    'pseudopotentials': {'Al': 'Al.pbe-n-kjpaw_psl.1.0.0.UPF'},
    'tstress': True,
    'tprnfor': True
}

results_relax = run_pw(orm.StructureData(ase=atoms), inputs)
scaled_structures = generate_scaled_structures(results_relax['structure'], scaling_factors)

inputs['input_data']['calculation'] = 'scf'
results = []

for label, scaled_structure in scaled_structures.items():
    results_scf = run_pw(scaled_structure, inputs)
    results.extend(results_scf.values())

plot = plot_energy_volume_curve(*results)

TypeError: issubclass() arg 1 must be a class

Full provenance is captured that can be easily represented graphically:

In [54]:
from aiida.tools.visualization import Graph
graph = Graph()
graph.recurse_ancestors(plot)
filename_written = graph.graphviz.render(outfile='provenance.png', format='png', view=True, cleanup=True)


(eog:32024): dconf-WARNING **: 23:08:06.433: failed to commit changes to dconf: Error spawning command line “dbus-launch --autolaunch=622c1c27dd18a85d8cb051f96335e825 --binary-syntax --close-stderr”: Child process exited with code 1

(eog:32024): dconf-WARNING **: 23:08:06.433: failed to commit changes to dconf: Error spawning command line “dbus-launch --autolaunch=622c1c27dd18a85d8cb051f96335e825 --binary-syntax --close-stderr”: Child process exited with code 1

(eog:32024): EOG-WARNING **: 23:08:06.458: Couldn't load icon: Icon 'image-loading' not present in theme Adwaita


### Submission to an HPC / Check pointing / Error handling
The Quantum ESPRESSO calculations can easily be submitted to a remote computer that is accessible over SSH. First, the target computer has to be represented through AiiDA's `Computer` model which is then configured to allow opening an SSH connection:

In [62]:
from aiida.orm import Computer, load_computer

try:
    computer = load_computer('some-computer')
except:
    computer = Computer(
        label='some-computer',
        hostname='localhost',
        transport_type='core.ssh',
        scheduler_type='core.direct',  # Can also use ``core.slurm`` for a SLURM scheduler for example
        workdir='/tmp',
    )

computer.configure(username='sph')

The `Computer` instance can then simply be passed in the `metadata.options.computer` input of the `launch_shell_job` call.

### Data Storage / Data Sharing
All data is automatically captured in the provenance graph that is stored in a performant relational database and object storage. The complete workflow can be exported to a zip file using:

In [55]:
from aiida.tools.archive import create_archive
create_archive([plot], 'archive.aiida')

07/19/2024 11:15:09 PM <7982> aiida.export: [REPORT] 
Archive Parameters
--------------------  -------------
Path                  archive.aiida
Version               main_0001
Compression           6

Inclusion rules
----------------------------  --------
Computers/Nodes/Groups/Users  Selected
Computer Authinfos            False
Node Comments                 True
Node Logs                     True

Traversal rules
---------------------------------  -----
Follow links input calc forwards   False
Follow links input calc backwards  True
Follow links create forwards       True
Follow links create backwards      True
Follow links return forwards       True
Follow links return backwards      False
Follow links input work forwards   False
Follow links input work backwards  True
Follow links call calc forwards    True
Follow links call calc backwards   True
Follow links call work forwards    True
Follow links call work backwards   True

07/19/2024 11:15:09 PM <7982> aiida.export: [REPORT] Val

PosixPath('archive.aiida')

The archive can be imported into any other AiiDA installation with a single command `verdi archive import archive.aiida` allowing the data to be explored.

### Publication of the workflow
To be added...